In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')
!mkdir dataset
!unzip /content/aksharantar_sampled.zip -d dataset

mkdir: cannot create directory ‘dataset’: File exists
Archive:  /content/aksharantar_sampled.zip
   creating: dataset/aksharantar_sampled/
   creating: dataset/aksharantar_sampled/asm/
  inflating: dataset/aksharantar_sampled/asm/asm_test.csv  
  inflating: dataset/aksharantar_sampled/asm/asm_train.csv  
  inflating: dataset/aksharantar_sampled/asm/asm_valid.csv  
   creating: dataset/aksharantar_sampled/ben/
  inflating: dataset/aksharantar_sampled/ben/ben_test.csv  
  inflating: dataset/aksharantar_sampled/ben/ben_train.csv  
  inflating: dataset/aksharantar_sampled/ben/ben_valid.csv  
   creating: dataset/aksharantar_sampled/brx/
  inflating: dataset/aksharantar_sampled/brx/brx_test.csv  
  inflating: dataset/aksharantar_sampled/brx/brx_train.csv  
  inflating: dataset/aksharantar_sampled/brx/brx_valid.csv  
   creating: dataset/aksharantar_sampled/guj/
  inflating: dataset/aksharantar_sampled/guj/guj_test.csv  
  inflating: dataset/aksharantar_sampled/guj/guj_train.csv  
  inflatin

In [ ]:
!cd dataset
!pwd


/content


In [4]:
import csv

def read_dataset(file_path):
    pairs = []
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            romanized_str = row[0].strip()
            devanagari_str = row[1].strip()
            romanized_str='\t'+romanized_str+'\n'
            devanagari_str='\t'+devanagari_str+'\n'
            pairs.append((romanized_str, devanagari_str))
    return pairs

In [5]:
file_path='/content/dataset/aksharantar_sampled/hin/hin_train.csv'
pairs=read_dataset(file_path)

In [6]:
file_path='/content/dataset/aksharantar_sampled/hin/hin_valid.csv'
valid_pairs=read_dataset(file_path)
file_path='/content/dataset/aksharantar_sampled/hin/hin_test.csv'
test_pairs=read_dataset(file_path)

In [7]:
def fun(pair):
  return len(pair[0])

pairs=sorted(pairs,key=fun)

['\t', 'श', 'स', '्', 'त', '्', 'र', 'ा', 'ग', 'ा', 'र', '\n']

In [ ]:
t=set()
t=t.union(set(list(pairs[0][1])))

In [ ]:
t

{'ग', 'त', 'र', 'श', 'स', 'ा', '्'}

In [ ]:
len(pairs)

51200

In [ ]:
list(pairs[0][0])

['\t', 's', 'h', 'a', 's', 't', 'r', 'a', 'g', 'a', 'a', 'r', '\n']

In [8]:
SOS_token = '\t'
EOS_token = '\n'


In [9]:
native_words=[]
devgiri_words=[]

def prepare_vocabulary(pairs):

  native_character_vocabulary=set()
  latin_character_vocabulary=set()

  native_character_vocabulary.add('\t')
  latin_character_vocabulary.add('\t')
  native_character_vocabulary.add('\n')
  latin_character_vocabulary.add('\n')

  native_enc_map={}
  latin_enc_map={}
  for (x,y) in pairs:
    native_character_vocabulary=native_character_vocabulary.union(set(list(x)))
    latin_character_vocabulary=latin_character_vocabulary.union(set(list(y)))
    

  native_character_vocabulary=sorted(list(native_character_vocabulary))
  latin_character_vocabulary=sorted(list(latin_character_vocabulary))

  

  i=0
  #native_char_map[SOS_token]=0
  #native_char_map[EOS_token]=0


  for ch in native_character_vocabulary:
    native_enc_map[ch]=i
    i+=1
  
  #native_char_map[EOS_token]=i
  
  #latin_char_map[SOS_token]=0
  native_dec_map = {v: k for k, v in native_enc_map.items()}

  i=0
  for ch in latin_character_vocabulary:
    latin_enc_map[ch]=i
    i+=1

  #latin_char_map[EOS_token]=i
  latin_dec_map = {v: k for k, v in latin_enc_map.items()}

  return native_character_vocabulary,latin_character_vocabulary,native_enc_map,latin_enc_map,native_dec_map,latin_dec_map,pairs


native_character_vocabulary,latin_character_vocabulary,native_enc_map,latin_enc_map,native_dec_map,latin_dec_map,pairs=prepare_vocabulary(pairs)






In [ ]:
for x,y in pairs:
  print(len(x),x)
  break

3 	j



In [10]:
max_encoder_seq_length = max([len(x) for x,y in pairs])
max_decoder_seq_length = max([len(y) for x,y in pairs])


In [11]:
print(max_encoder_seq_length)
print(max_decoder_seq_length)

26
22


In [12]:
import numpy as np
from torch.nn.utils.rnn import pad_sequence
def prepare_encoding(input,native_char_map,latin_char_map,batch_size):
  #input_encoding = np.zeros((M, max_encoder_seq_length, num_input_tokens + 1), dtype="float")
  input_encoding=[torch.tensor([native_char_map[ch] for ch in train_pair[0]],device=device) for train_pair in input]
  #input_encoding.append(EOS_token)
  input_encoding = pad_sequence(input_encoding, batch_first=True)
  input_encoding=torch.tensor(input_encoding).to(device)
  target_encoding=[torch.tensor([latin_char_map[ch] for ch in train_pair[1]],device=device)for train_pair in input]
  #target_encoding.append(EOS_token)
  target_encoding = pad_sequence(target_encoding, batch_first=True)
  target_encoding=torch.tensor(target_encoding).to(device)

  return input_encoding,target_encoding

input_encoding,target_encoding=prepare_encoding(pairs[0:10],native_enc_map,latin_enc_map,10)


<ipython-input-12-4ff56a6688cc>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_encoding=torch.tensor(input_encoding).to(device)
<ipython-input-12-4ff56a6688cc>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_encoding=torch.tensor(target_encoding).to(device)


In [11]:
latin_enc_map

{'\t': 0,
 '\n': 1,
 'ँ': 2,
 'ं': 3,
 'ः': 4,
 'अ': 5,
 'आ': 6,
 'इ': 7,
 'ई': 8,
 'उ': 9,
 'ऊ': 10,
 'ऋ': 11,
 'ए': 12,
 'ऐ': 13,
 'ऑ': 14,
 'ओ': 15,
 'औ': 16,
 'क': 17,
 'ख': 18,
 'ग': 19,
 'घ': 20,
 'ङ': 21,
 'च': 22,
 'छ': 23,
 'ज': 24,
 'झ': 25,
 'ञ': 26,
 'ट': 27,
 'ठ': 28,
 'ड': 29,
 'ढ': 30,
 'ण': 31,
 'त': 32,
 'थ': 33,
 'द': 34,
 'ध': 35,
 'न': 36,
 'प': 37,
 'फ': 38,
 'ब': 39,
 'भ': 40,
 'म': 41,
 'य': 42,
 'र': 43,
 'ल': 44,
 'ळ': 45,
 'व': 46,
 'श': 47,
 'ष': 48,
 'स': 49,
 'ह': 50,
 '़': 51,
 'ऽ': 52,
 'ा': 53,
 'ि': 54,
 'ी': 55,
 'ु': 56,
 'ू': 57,
 'ृ': 58,
 'ॅ': 59,
 'े': 60,
 'ै': 61,
 'ॉ': 62,
 'ो': 63,
 'ौ': 64,
 '्': 65}

In [12]:
pairs

[('\tshastragaar\n', '\tशस्त्रागार\n'),
 ('\tbindhya\n', '\tबिन्द्या\n'),
 ('\tkirankant\n', '\tकिरणकांत\n'),
 ('\tyagyopaveet\n', '\tयज्ञोपवीत\n'),
 ('\tratania\n', '\tरटानिया\n'),
 ('\tvaganyache\n', '\tवागण्याचे\n'),
 ('\tdeshbharamadhye\n', '\tदेशभरामध्ये\n'),
 ('\tsughadpan\n', '\tसुघड़पन\n'),
 ('\tmohiwal\n', '\tमोहीवाल\n'),
 ('\tsarvasangrah\n', '\tसर्वसंग्रह\n'),
 ('\tbaseko\n', '\tबसेको\n'),
 ('\ttumchyapaiki\n', '\tतुमच्यापैकी\n'),
 ('\tkanyakubja\n', '\tकान्यकुब्ज\n'),
 ('\tintoxination\n', '\tइनटॉक्सिनेशन\n'),
 ('\tmaturity\n', '\tमेच्यूरिटी\n'),
 ('\tagri\n', '\tअगरी\n'),
 ('\tanukramanupat\n', '\tअनुक्रमानुपात\n'),
 ('\tdhulchand\n', '\tधूलचन्द\n'),
 ('\tavleha\n', '\tअवलेह\n'),
 ('\tabroard\n', '\tएबरोर्ड\n'),
 ('\tballers\n', '\tबैलर्स\n'),
 ('\tbavli\n', '\tबार्गली\n'),
 ('\tpunctured\n', '\tपंक्चर्ड\n'),
 ('\thankers\n', '\tहैंकर्स\n'),
 ('\tjawanonke\n', '\tजवानोंके\n'),
 ('\tpainki\n', '\tपौंकी\n'),
 ('\tjagandham\n', '\tजगनधाम\n'),
 ('\tponnium\n', '\tपोन्नियम\n'),

In [21]:
input_encoding=input_encoding.T

In [19]:
target_encoding=target_encoding.T

In [20]:
target_encoding

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [24, 46, 12,  6,  7, 15, 24, 49, 15, 12],
        [ 1, 55, 38, 43, 50,  6, 55, 55, 19, 36],
        [ 0,  1, 29, 12,  1,  8, 27, 29, 65, 46],
        [ 0,  0, 55, 49,  0,  1, 55, 55,  1, 55],
        [ 0,  0,  1,  1,  0,  0,  1,  1,  0,  1]], device='cuda:0')

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext

# Define the encoder RNN
class EncoderRNN(nn.Module):
    def __init__(self, input_size,embedding_size, hidden_size, num_layers, cell_type,p):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout=nn.Dropout(p)
        self.cell_type=cell_type

        self.embedding = nn.Embedding(input_size, embedding_size)
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers,dropout=p)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers,dropout=p)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers,dropout=p)

    def forward(self, input):
      
       
        embedded = self.dropout(self.embedding(input))
        #print(embedded.shape)
        if(self.cell_type=='LSTM'):
          outputs,(hidden,cell)= self.rnn(embedded)
          return (hidden,cell)

        else:
          outputs,hidden=self.rnn(embedded)
          return hidden

    


# Define the decoder RNN
class DecoderRNN(nn.Module):
    def __init__(self, input_size,embedding_size, hidden_size, output_size, num_layers, cell_type,p):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type=cell_type
        self.dropout=nn.Dropout(p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers,dropout=p)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers,dropout=p)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers,dropout=p)
        #self.fc = nn.Linear(hidden_size, output_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input, hidden,cell=None):
        
        input=input.unsqueeze(0)
        output = self.dropout(self.embedding(input))
        #output = F.relu(output)
        #print(output.shape)
        if(self.cell_type=='LSTM'):
          outputs, (hidden,cell) = self.rnn(output, (hidden,cell))
          predictions=self.out(outputs)
          predictions=predictions.squeeze(0)
          #output = self.softmax(output)
          return predictions, hidden,cell

        else:
          outputs,hidden=self.rnn(output,hidden)
          predictions=self.out(outputs)
          predictions=predictions.squeeze(0)
          return predictions,hidden


  



#encoder1 = EncoderRNN(len(native_character_vocabulary), 256,1,'RNN',10).to(device)
#decoder1=DecoderRNN(len(latin_character_vocabulary),256,1,'RNN',10).to(device)



In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext

# Define the encoder RNN
class Encodermodel(nn.Module):
  
    def __init__(self, n_input,n_embedding, n_hidden, n_layers, cell_type,p):
        super(Encodermodel, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.dropout=nn.Dropout(p)
        self.cell_type=cell_type

        self.embedding_layer = nn.Embedding(n_input, n_embedding)
        if cell_type == 'RNN':
            self.rnn_cell = nn.RNN(n_embedding, n_hidden, n_layers,dropout=p)
        elif cell_type == 'LSTM':
            self.rnn_cell = nn.LSTM(n_embedding, n_hidden, n_layers,dropout=p)
        elif cell_type == 'GRU':
            self.rnn_cell = nn.GRU(n_embedding, n_hidden, n_layers,dropout=p)

    def forward(self, x):
      
       
        embedded_x = self.dropout(self.embedding_layer(x))
        #print(embedded.shape)
        if(self.cell_type=='LSTM'):
          outputs,(hidden,cell)= self.rnn_cell(embedded_x)
          return (hidden,cell)

        else:
          outputs,hidden=self.rnn_cell(embedded_x)
          return hidden

    


# Define the decoder RNN
class Decodermodel(nn.Module):
    def __init__(self, n_input,n_embedding, n_hidden, n_output, n_layers, cell_type,p):
        super(Decodermodel, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.cell_type=cell_type
        self.dropout=nn.Dropout(p)

        self.embedding_layer = nn.Embedding(n_input, n_embedding)
        if cell_type == 'RNN':
            self.rnn_layer = nn.RNN(n_embedding, n_hidden, n_layers,dropout=p)
        elif cell_type == 'LSTM':
            self.rnn_layer = nn.LSTM(n_embedding, n_hidden, n_layers,dropout=p)
        elif cell_type == 'GRU':
            self.rnn_layer = nn.GRU(n_embedding, n_hidden, n_layers,dropout=p)
        #self.fc = nn.Linear(hidden_size, output_size)
        self.out = nn.Linear(n_hidden, n_output)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x, hidden,cell=None):
        
        x=x.unsqueeze(0)
        output = self.dropout(self.embedding_layer(x))
        #output = F.relu(output)
        #print(output.shape)
        if(self.cell_type=='LSTM'):
          outputs, (hidden,cell) = self.rnn_layer(output, (hidden,cell))
          predictions=self.out(outputs)
          predictions=predictions.squeeze(0)
          #output = self.softmax(output)
          return predictions, hidden,cell

        else:
          outputs,hidden=self.rnn_layer(output,hidden)
          predictions=self.out(outputs)
          predictions=predictions.squeeze(0)
          return predictions,hidden


  



#encoder1 = EncoderRNN(len(native_character_vocabulary), 256,1,'RNN',10).to(device)
#decoder1=DecoderRNN(len(latin_character_vocabulary),256,1,'RNN',10).to(device)



In [ ]:
len(pairs[0:10])

10

In [ ]:
len(latin_character_vocabulary)

66

In [15]:
from torch.types import Device
class EncDecModel(nn.Module):
  def __init__(self,encoder,decoder):
    super(EncDecModel,self).__init__()
    self.encoder=encoder
    self.decoder=decoder

  def forward(self,X,Y,teacher_force_ratio=0.5):
    batch_size=X.shape[1]
    out_len=Y.shape[0]
    out_vocab_size=len(latin_character_vocabulary)
    ans=["" for i in range (0,batch_size)]
    outputs=torch.zeros(out_len,batch_size,out_vocab_size).to(device)


    if(self.encoder.cell_type=='LSTM'):
      hidden,cell=self.encoder(X)
      
      x=Y[0]
      for t in range(1,out_len):
        output,hidden,cell=self.decoder(x,hidden,cell)
        outputs[t]=output

        max_y=output.argmax(1)
        print(y[t],"hu",max_y)
        x=y[t] if random.random()<teacher_force_ratio else max_y

    else:
      hidden=self.encoder(X)
      #outputs=torch.zeros(target_len,batch_size,target_vocab_size).to(device)

      x=Y[0]
      for t in range(1,out_len):
        output,hidden=self.decoder(x,hidden)
        outputs[t]=output

        max_y=output.argmax(1)
        for j in range(batch_size):
          ind=max_y[j].item()
          ans[j]+=latin_dec_map[ind]
        #print(best_guess)
        #print(target[t],"hu",best_guess)
        x=Y[t] if random.random()<teacher_force_ratio else max_y

    #print(ans)
    #print(outputs.shape)
    #print(target_len)
    return outputs,ans

epochs=2
lr=0.001
batch_size=64

load_model=False
input_size_encoder=len(native_character_vocabulary)
input_size_decoder=len(latin_character_vocabulary)

output_size=len(latin_character_vocabulary)
encoder_embedding_size=300
decoder_embedding_size=300
hidden_size=256
num_layers=2
enc_dropout=0.5

dec_dropout=0.5

#writer=SummaryWriter('fruns/loss_plot')



encoder_net=Encodermodel(input_size_encoder,encoder_embedding_size,hidden_size,num_layers,'RNN',enc_dropout).to(device)
decoder_net=Decodermodel(input_size_decoder, decoder_embedding_size, hidden_size, output_size, num_layers, 'RNN', dec_dropout).to(device)

model=EncDecModel(encoder_net,decoder_net).to(device)

optimizer=optim.Adam(model.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()


def train(epochs,model,batch_size):

  for epoch in range(epochs):
    loss=0
    count=0
    tot_ans=[]
    for i in range(0,len(pairs),batch_size):
    #for i in range(0,10,batch_size):
      input_encoding,target_encoding=prepare_encoding(pairs[i:i+batch_size],native_enc_map,latin_enc_map,batch_size)
      #input_encoding,target_encoding=prepare_encoding(pairs[0:10],native_enc_map,latin_enc_map,batch_size)
      input_encoding=input_encoding.T
      target_encoding=target_encoding.T
      inp_data=input_encoding
      target=target_encoding
      #print(target)
      optimizer.zero_grad()
      output,ans=model(inp_data,target)
      tot_ans.append(ans)

      #ans=prepare_decoding(output,latin_dec_map,target.shape[0],batch_size)
      #out = torch.argmax(output, dim=2)
      #print(out.shape)
      #out = out.squeeze(1)
      #tokens=[for token in out]
      #out = [latin_dec_map[token.item()] for token in out]
      #print(output)


      output=output[1:].reshape(-1,output.shape[2])
      
      #print(output,"hii")
      
      target=target[1:].reshape(-1)
      #print(target)

      optimizer.zero_grad()
      loss=criterion(output,target)
      count+=len(pairs[i:i+batch_size])
      #print(loss)
      torch.autograd.set_detect_anomaly(True)
      loss.backward(retain_graph=True)
      torch.nn.utils.clip_grad_norm(model.parameters(),max_norm=1)
      optimizer.step()
    print(loss)
  return tot_ans

tot_ans=train(5,model,batch_size)


<ipython-input-12-4ff56a6688cc>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_encoding=torch.tensor(input_encoding).to(device)
<ipython-input-12-4ff56a6688cc>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_encoding=torch.tensor(target_encoding).to(device)
<ipython-input-15-bfc8470bb563>:122: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(),max_norm=1)


tensor(2.7840, grad_fn=<NllLossBackward0>)
tensor(3.0102, grad_fn=<NllLossBackward0>)


KeyboardInterrupt: ignored

In [33]:
tot_ans

[['\n़़ीहीॉधछङौल',
  'नःंतफंगप़ोफछ',
  'िॉऐयलयवनफयऔढ',
  '़हऔौपःसओभढकॉ',
  '़िकउपंोङँछँँ',
  'धछरऊहँंथभसष़',
  'शंदीथशडः्ङशष',
  'औऐइू\tकेपध\nधँ',
  'तोभलटतषटँेउइ',
  'ंकखह़तो़औईखग',
  'छरएद\tँृडेोंो',
  'ँॉशओऋनलूःईफय',
  'ँांज़िेृँढँफ',
  '़वणइतमवभ्छफ\t',
  'इफवउृोभीधोधआ',
  'ँवमआऐैँजँईनँ',
  '\nोऔैलमलथफखिङ',
  'िऽइडकभोऑछल़ि',
  'फखाःडयईञशभषँ',
  'फोृूवहअझऔउेँ',
  'णृफिळैऽभसृधग',
  'ँॉफजकङडौनल़फ',
  'ँरओॉङिधध़मफब',
  'एवढृोीटईिप़ँ',
  'ँऐेऴकफखफो\nँ',
  'ँतंॉृाछलफ\tँस',
  'ङओूडआऐऔऐआफनल',
  'गणथिअऋऑदँसँँ',
  'िलजंञेङुभिखस',
  'फवपौळँऑऊआषफध',
  'ँोअटतशमभ़झधह',
  'खंओोवॉडउफरँॉ',
  'औऽधृऽऔटउआधँष',
  'ँंँळोफैपऽऔशक',
  'घदझिऔखॉकभऽपऔ',
  'एइरैनलयइफरण\n',
  '़ाूप\n्फहहफिृ',
  'ँोउयूैनुऔचेँ',
  'भृपभहढचॉिअटऔ',
  'ो़ओअछषॉभैइंफ',
  'ै\tंँनँबंऽँभभ',
  'ॅो़े\nकिूसदृध',
  'नौठऔकूृऽटटऔो',
  'औयएणइपणखंलफ\n',
  'ँ़टो़बवटृइँऔ',
  'ीभीधथॅॉॉफखइश',
  'ँॉऐःकइय\nेवेँ',
  'भढउऔ्ँॉझसटशल',
  'ँछवीहअःिफँँ़',
  '़ईूगषफॉेँढपॉ',
  '़इःःडहईजऔवणँ',
  'ँकतलंझठऑृधधछ',
  'भषउऋसतदअंलभन',
  'ङझएेङअशङङईशध',
  'फकहचएऔडळकऑन

In [ ]:
def calc_accuracy(predicted,actual):
  sum=0
  count=0
  for i in range(predicted.shape):
    for j in range(len(predicted[i])):
      if(predicted[i][j]=='\n'):
        break
      else:
        if(predicted[i][j]==actual[i][j]):
          count+=1
      



In [ ]:
def clean_output(tot_ans):
  tot=[]
  for s in tot_ans:
    #for s in tot_ans[i]:
      #s=tot_ans[i]
      #print(s)
      temp=""
      for ch in s:
        print(ch)
        if(ch!='\t' or ch!='\n'):
          temp+=ch
        if(ch=='\n'):
          break

          
      tot.append(temp)

  return tot
      


In [ ]:
print(clean_output(tot_ans))

ज
			
वी
		
एफडी

आरएस

इह
		
ओआई
	
जीटी

सीडी

ओग्
	
एनवी

['ज\n\t\t\tवी\n\t\tएफडी\nआरएस\nइह\n\t\tओआई\n\tजीटी\nसीडी\nओग्\n\tएनवी\n']


In [ ]:
pairs[0:10]

[('\tj\n', '\tज\n'),
 ('\tv\n', '\tवी\n'),
 ('\tfd\n', '\tएफडी\n'),
 ('\trs\n', '\tआरएस\n'),
 ('\tih\n', '\tइह\n'),
 ('\toi\n', '\tओआई\n'),
 ('\tgt\n', '\tजीटी\n'),
 ('\tcd\n', '\tसीडी\n'),
 ('\tog\n', '\tओग्\n'),
 ('\tnv\n', '\tएनवी\n')]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext

# Define the encoder RNN
class EncoderRNN(nn.Module):
    def __init__(self, input_size,embedding_size, hidden_size, num_layers, cell_type,p,isbidirectional=True):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout=nn.Dropout(p)
        self.cell_type=cell_type

        self.embedding = nn.Embedding(input_size, embedding_size)
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers,bidirectional=isbidirectional,dropout=p)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers,bidirectional=isbidirectional,dropout=p)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers,bidirectional=isbidirectional,dropout=p)

        self.fc_hidden=nn.Linear(hidden_size*2,hidden_size)
        self.fc_cell=nn.Linear(hidden_size*2,hidden_size)

    def forward(self, input):
      
       
        embedded = self.dropout(self.embedding(input))
        #print(embedded.shape)
        if(self.cell_type=='LSTM'):
          outputs,(hidden,cell)= self.rnn(embedded)
          hidden=self.fc_hidden(torch.cat((hidden[0:1],hidden[1:2]),dim=2))
          cell=self.fc_cell(torch.cat((cell[0:1],cell[1:2]),dim=2))
          return (outputs,hidden,cell)

        else:
          outputs,hidden=self.rnn(embedded)
          hidden=self.fc_hidden(torch.cat((hidden[0:1],hidden[1:2]),dim=2))

          return outputs,hidden

    


# Define the decoder RNN
class DecoderRNN(nn.Module):
    def __init__(self, input_size,embedding_size, hidden_size, output_size, num_layers, cell_type,p):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type=cell_type
        self.dropout=nn.Dropout(p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        if cell_type == 'RNN':
            self.rnn = nn.RNN(hidden_size*2+embedding_size, hidden_size, num_layers)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers)

        self.energy=nn.Linear(hidden_size*3,1)
        self.softmax = nn.Softmax(dim=0)
        self.relu=nn.ReLU()
        #self.fc = nn.Linear(hidden_size, output_size)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, states_enc,hidden,cell=None):
        
        input=input.unsqueeze(0)
        output = self.dropout(self.embedding(input))
        #output = F.relu(output)
        #print(output.shape)
        len=states_enc.shape[0]
        h_reshaped=hidden.repeat(len,1,1)
        energy=self.relu(self.energy(torch.cat((h_reshaped,states_enc),dim=2)))
        attention=self.softmax(energy)

        attention=attention.permute(1,2,0)
        states_enc=states_enc.permute(1,0,2)

        context_vector=torch.bmm(attention,states_enc).permute(1,0,2)

        rnn_input=torch.cat((context_vector,output),dim=2)
        if(self.cell_type=='LSTM'):
          outputs, (hidden,cell) = self.rnn(rnn_input, (hidden,cell))
          predictions=self.out(outputs)
          predictions=predictions.squeeze(0)
          #output = self.softmax(output)
          return predictions, hidden,cell

        else:
          outputs,hidden=self.rnn(rnn_input,hidden)
          predictions=self.out(outputs)
          predictions=predictions.squeeze(0)
          return predictions,hidden


  



#encoder1 = EncoderRNN(len(native_character_vocabulary), 256,1,'RNN',10).to(device)
#decoder1=DecoderRNN(len(latin_character_vocabulary),256,1,'RNN',10).to(device)

from torch.types import Device
class Seq2seq(nn.Module):
  def __init__(self,encoder,decoder):
    super(Seq2seq,self).__init__()
    self.encoder=encoder
    self.decoder=decoder

  def forward(self,source,target,teacher_force_ratio=0.5):
    batch_size=source.shape[1]
    target_len=target.shape[0]
    target_vocab_size=len(latin_character_vocabulary)
    ans=["" for i in range (0,batch_size)]
    outputs=torch.zeros(target_len,batch_size,target_vocab_size).to(device)


    if(self.encoder.cell_type=='LSTM'):
      enc_states,hidden,cell=self.encoder(source)
      
      x=target[0]
      for t in range(1,target_len):
        output,hidden,cell=self.decoder(x,enc_states,hidden,cell)
        outputs[t]=output

        best_guess=output.argmax(1)
        print(target[t],"hu",best_guess)
        x=target[t] if random.random()<teacher_force_ratio else best_guess

    else:
      enc_states,hidden=self.encoder(source)
      #outputs=torch.zeros(target_len,batch_size,target_vocab_size).to(device)

      x=target[0]
      for t in range(1,target_len):
        output,hidden=self.decoder(x,enc_states,hidden)
        outputs[t]=output

        best_guess=output.argmax(1)
        for j in range(batch_size):
          ind=best_guess[j].item()
          ans[j]+=latin_dec_map[ind]
        #print(best_guess)
        #print(target[t],"hu",best_guess)
        x=target[t] if random.random()<teacher_force_ratio else best_guess

    #print(ans)
    #print(outputs.shape)
    #print(target_len)
    return outputs,ans

epochs=2
lr=0.001
batch_size=10

load_model=False
input_size_encoder=len(native_character_vocabulary)
input_size_decoder=len(latin_character_vocabulary)

output_size=len(latin_character_vocabulary)
encoder_embedding_size=300
decoder_embedding_size=300
hidden_size=1024
num_layers=1
enc_dropout=0.5

dec_dropout=0.5

#writer=SummaryWriter('fruns/loss_plot')



encoder_net=EncoderRNN(input_size_encoder,encoder_embedding_size,hidden_size,num_layers,'RNN',enc_dropout).to(device)
decoder_net=DecoderRNN(input_size_decoder, decoder_embedding_size, hidden_size, output_size, num_layers, 'RNN', dec_dropout).to(device)

model=Seq2seq(encoder_net,decoder_net).to(device)
optimizer=optim.Adam(model.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()


def train(epochs,model,batch_size):

  for epoch in range(10):
    loss=0
    count=0
    tot_ans=[]
    #for i in range(0,len(pairs),batch_size):
    for i in range(0,10,batch_size):
      #input_encoding,target_encoding=prepare_encoding(pairs[i:i+batch_size],native_enc_map,latin_enc_map,batch_size)
      input_encoding,target_encoding=prepare_encoding(pairs[0:10],native_enc_map,latin_enc_map,batch_size)
      input_encoding=input_encoding.T
      target_encoding=target_encoding.T
      inp_data=input_encoding
      target=target_encoding
      #print(target)
      optimizer.zero_grad()
      output,ans=model(inp_data,target)
      tot_ans.append(ans)

      #ans=prepare_decoding(output,latin_dec_map,target.shape[0],batch_size)
      #out = torch.argmax(output, dim=2)
      #print(out.shape)
      #out = out.squeeze(1)
      #tokens=[for token in out]
      #out = [latin_dec_map[token.item()] for token in out]
      #print(output)


      output=output[1:].reshape(-1,output.shape[2])
      
      #print(output,"hii")
      
      target=target[1:].reshape(-1)
      #print(target)

      optimizer.zero_grad()
      loss=criterion(output,target)
      count+=len(pairs[i:i+batch_size])
      print(loss)
      torch.autograd.set_detect_anomaly(True)
      loss.backward(retain_graph=True)
      torch.nn.utils.clip_grad_norm(model.parameters(),max_norm=1)
      optimizer.step()

  return tot_ans

tot_ans=train(1,model,batch_size)



<ipython-input-12-4ff56a6688cc>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_encoding=torch.tensor(input_encoding).to(device)
<ipython-input-12-4ff56a6688cc>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_encoding=torch.tensor(target_encoding).to(device)
<ipython-input-37-2773a6591399>:227: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(),max_norm=1)


tensor(4.2162, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.4406, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.9567, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5809, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3531, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7191, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3336, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1891, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0933, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0669, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
tot_ans

[['ज\n\t\t\t',
  'वी\n\t\t',
  'एफडी\n',
  'आरएस\n',
  'इह\n\t\t',
  'ओआई\n\t',
  'जीटी\n',
  'सीडी\n',
  'ओग्\n\t',
  'एनवी\n']]